In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
from tqdm import tqdm
import time
import matplotlib.gridspec as gridspec
import datetime
import h5py
import glob
import os
import scipy.signal
import cv2
import math
from tensorflow.keras.models import Sequential, model_from_json

In [2]:
#load raw 30 freq model 
json_file = open('/scratch/gpfs/kx2561/ML/rawSpecModels/rawSpec_30f/newest_MLPmodel_30freq.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('/scratch/gpfs/kx2561/ML/rawSpecModels/rawSpec_30f/newest_MLPmodel_30freq.h5')
print("Loaded model from disk")

Loaded model from disk


2023-09-04 23:01:10.102492: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30178 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0004:04:00.0, compute capability: 7.0
2023-09-04 23:01:10.103805: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 26281 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0035:03:00.0, compute capability: 7.0


In [3]:
# postprocessing parameters-- vary for different models
lookBefore = 5   # check if a 1 exists in (lookBefore) places before and after
howLong = 250 # length of small chunks of 1s to remove
threshold = 0.378   # threshold in which the raw prediction is interpreted as a "1"/"Mode"
breakTime = 0.015  # break time between chunks of 1s at the end of postprocessing (link together the full temporal presence of the mode)

# get all times (for one spectrogram). Can also just load shot with Sxx, Sxx_enhanced, f, t = loadShot(193727, 3)
df = pd.read_csv('/scratch/gpfs/kx2561/ML/rawSpecModels/rawSpec_30f/Times.csv')
t = df['Times']

# mode shots to look at (these channels specifically I had checked and were not in the training set)
modeShots = [194045, 194043, 194399, 194123, 193782, 193752,193754, 193727, 193843, 193806, 193775, 193782, 194050, 194128, 194128, 194467, 194468, 193807, 193716, 193691, 193691, 193690, 194048, 194048, 194052, 194399, 194046, 194047]
modeChns = [3, 4, 6, 4,6, 3, 4, 6, 4, 4, 5, 4,6,3, 4, 4, 3, 4, 4, 3, 5, 3, 3, 4, 5, 4, 4, 4]

# modeless shots to look at
modelessShots = [193806, 194370, 194009, 193691, 194371, 193731, 194042, 193995, 194370, 194370, 194385, 194371, 194371, 194470, 194009, 194051, 193685, 193807, 194279, 194468, 193695, 193695, 194081, 194128, 193782, 193843, 193752, 193754, 194049]
modelessChns = [7, 6, 8, 6, 6, 5, 7, 6, 6, 3, 3, 6, 4, 7, 5, 7, 5, 6, 5, 5, 6, 5, 7, 6, 7, 3, 7, 3, 3]

In [4]:
##### spectrogram enhancement ###############
def specgr (sig_in,spec_params,thr=0.9, gaussblr_win=(31,3)):
    f, t, Sxx = scipy.signal.spectrogram(sig_in, nperseg=spec_params['nperseg'], noverlap=spec_params['noverlap'],fs=spec_params['fs'], window=spec_params['window'],scaling=spec_params['scaling'], detrend=spec_params['detrend'])
    Sxx = np.log(Sxx + spec_params['eps']) #offset by epsilon to prevent log(0)
    Sxx=(Sxx-np.min(Sxx))/(np.max(Sxx)-np.min(Sxx)) #z score scaling
    Sxx = Sxx[:-1,:];f=f[:-1] 
    
    Sxx_enhanced = quantfilt(Sxx,thr) 
    Sxx_enhanced = gaussblr(Sxx_enhanced,gaussblr_win)
    Sxx_enhanced = meansub(Sxx_enhanced)    
    Sxx_enhanced = morph(Sxx_enhanced)
    Sxx_enhanced = meansub(Sxx_enhanced)    
    return Sxx,Sxx_enhanced,f/1000,t


def norm(data):
    mn = data.mean()
    std = data.std()
    return((data-mn)/std)

def rescale(data):
    return (data-data.min())/(data.max()-data.min())

def quantfilt(src,thr=0.9):
    filt = np.quantile(src,thr,axis=0)
    out = np.where(src<filt,0,src)
    return out

# gaussian filtering (just get middle part, filter out highs and lows)
def gaussblr(src,filt=(31, 3)):
    src = (rescale(src)*255).astype('uint8')
    out = cv2.GaussianBlur(src,filt,0)
    return rescale(out)

# mean filtering
def meansub(src):
    mn = np.mean(src,axis=1)[:,np.newaxis]
    out = np.absolute(src - mn)
    return rescale(out)

# morphological filtering
def morph(src):
    src = (rescale(src)*255).astype('uint8')
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    mask = cv2.morphologyEx(src, cv2.MORPH_CLOSE, se1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
    return rescale(mask)

spec_params={
    'nperseg': 800, # default 1024
    'noverlap': 550, # default: nperseg / 4
    'fs': 500000, # raw signal sample rate is 4MHz
    'window': 'hamm',
    'scaling': 'density', # {'density', 'spectrum'}
    'detrend': 'linear', # {'linear', 'constant', False}
    'eps': 1e-11} 

def loadShot(shotn,ece_chn):
    # data_spec = pickle.load(open(os.path.join('/projects/EKOLEMEN/negd_ece/negd_spec/','%d.pkl' % (shotn)),'rb'))
    #rb is read binary 
    data_raw = pickle.load(open(os.path.join('/projects/EKOLEMEN/negd_ece/negd_raw/','%d.pkl' % (shotn)),'rb')) 
    return specgr (data_raw[f'ECEVS{ece_chn:02d}'],spec_params)

In [23]:
# evaluate model on shots (look at performance)
def makePrediction(loaded_model, spec):
    predictions = []
    
    for i in range(spec.shape[-1]):
        make_pred = loaded_model.predict(spec[:30, i].reshape(1, 30))  # first 30 frequencies of raw spec
        predictions.append(make_pred[0][0])
            
    return predictions

In [6]:
##### model postprocessing algorithm begins here ###########
# Step 1: turn raw predictions into series of 0s and 1s (1 if past a certain threshold)
def processPredictions(a, threshold):
    newCol = []

    #parse through raw predictions
    for i in range(len(a)):
        if a[i] >= threshold:
            newCol.append(1)
        else:
            newCol.append(0)
        
    return newCol   # predictions in format of 0s and 1s

In [7]:
# Step 2: interpolation (look at previous and later predictions to turn 0s to 1s)
def filtering(column, lookBefore):
    newCol = column.copy()   # column of model predictions (0s or 1s)
    
    # move backwards through the raw predictions to check 
    for i in range(len(newCol)-1, -1, -1):
        difference = i - lookBefore    # lookBefore = how many previous values to look at
        if difference < 0:
            nStart = 0  # nstart = the first previous index to look at  (Ex. for i = 5 and lookBefore = 5, I'd look at 0, 1, 2, 3, and 4)
        else:
            nStart = difference  
    
        upperCheck = i + lookBefore + 1  # the last following index to look at (Ex. for i = 5 and lookBefore = 5, looking at 6, 7, 8, 9, 10)
        if upperCheck > len(column):
            upperCheck = len(column)
                
        # interpolation with upper and lower indices to check
        if newCol[i] == 0:
            seenBefore = False
            seenAfter = False
            for index in range(nStart, i):
                if newCol[index] == 1:
                    seenBefore = True   # if there is at least one "1" in the nPast indices before the "0"
            
            for j in range(i, upperCheck):
                if newCol[j] == 1:
                    seenAfter = True
            
            if seenBefore and seenAfter:  
                newCol[i] = 1
                
        # zeroes an isolated 1 if it's surrounded on both sides by zeroes (ex. 0 1 0 => 0 0 0)
        if newCol[i] == 1:
            if i == 0:
                if column[i+1] == 0:
                    newCol[i] = 0
            elif i == len(newCol)-1:
                if column[i-1] == 0:
                    newCol[i] = 0
            else:
                if newCol[i-1] == 0 and newCol[i+1] == 0:
                    newCol[i] = 0
                    
    return newCol

In [8]:
# Step 3: filter out small isolated chunks of 1s (limit randomness of predictions)
# threshold = the upper limit on the length of the isolated chunk (2 = 0.001 s, 3 = 0.0015 s)
def filteringOnes(arr, threshold):
    ones = []
    removeIndices = []  # which indices to turn from zeros to ones
    processedArr = arr.copy()
    
    for index in range(len(arr)):
        if arr[index] == 1:
            ones.append(index)
            
    indices = filtering2(ones, 1.00005)  # get locations of ones (ie if 1 at 3, and 1 at 4, it returns (3, 4)
    #print(indices)
    
    for i in indices:
        if (i[-1] - i[0]) <= threshold-1: # ex. threshold of 2 indicates a small burst of 0.001 s
            for val in range(i[0], i[-1]+1):
                removeIndices.append(val)   # add 
                
    for r in removeIndices:
        processedArr[r] = 0
    return processedArr

In [9]:
# Step 4: get start and end of mode (temporal presence)
def filtering2(col, breakTime):
    newCol = []
    start = 0
    end = 0  
    
    column = [i for i in col if i != 0]
    
    for i in range(1, len(column)):
        if column[i] != 'N/A':
            currentStart = column[i]
            currentEnd = column[i-1]
            
            if start == 0 and currentStart - currentEnd <= (breakTime + 0.00005):
                start = column[i-1]
                end = column[i]
            
            elif currentStart - end <= (breakTime + 0.00005):   # added to account for floating point inaccuracy
                end = column[i]
                
            elif (end - start) > 0:
                newCol.append((start, end))
                start = 0
                end = 0
            else:
                start = 0
                end = 0
                
        if i == len(column)-1:
            if end-start > 0:
                newCol.append((start, end))

    return newCol

In [30]:
# plot postprocessed predictions
# plot raw signal, spectrogram, enhanced spectrogram, and predictions v. time with rolling average line
# upperBound = upper boundary of frequency space (ie 30 => first 30 frequencies are plotted)
# times = predicted 1s of the model
# mode: False if shot is modeless, True if shot has mode
# raw predictions = array of model predictions on whole spectrogram
def plotECE(tGood, shotn, ece_chn, upperBound, times, rawPredictions, mode):
    data_raw = pickle.load(open(os.path.join('/projects/EKOLEMEN/negd_ece/negd_raw/','%d.pkl' % (shotn)),'rb')) 
    Sxx,Sxx_enhanced,f,t = loadShot(shotn, ece_chn)
    Sxx[upperBound:, :] = 0
    Sxx_enhanced[upperBound:, :] = 0
     
    fig = plt.figure(figsize=(10,8),dpi=100)
    
    grd = gridspec.GridSpec(7, 1)
    
   
    if mode: 
        a = pickle.load(open(f'/scratch/gpfs/aj17/datasets/NT-dataset/dataset/mode/shot-{shotn}-ece-{chn}.pkl', 'rb'))
        modeStart = a['Mode Start Time(s)']
        modeEnd = a['Mode End Time(s)'] 
    else:
        a = pickle.load(open(f'/scratch/gpfs/aj17/datasets/NT-dataset/dataset/nomode/shot-{shotn}-ece-{chn}.pkl', 'rb'))
    
    # raw ECE signal
    ax = fig.add_subplot(grd[0])
    ax.plot(data_raw['times'],data_raw[f'ECEVS{ece_chn:02d}'])
    _=plt.title('Shot# %i - ECE# %i' % (shotn,ece_chn))
    _=plt.xlim(data_raw['times'][0],data_raw['times'][-1])
    _=plt.ylim([-1,10])
    _=plt.ylabel('raw ece')

    # draw raw spectrogram for first (upperBound) frequencies
    ax = fig.add_subplot(grd[1:3])
    ax.pcolormesh(tGood,f[:upperBound],Sxx[:upperBound, tstart:tend],shading='auto', cmap='hot')
    
    # draw out prediction
    for i in range(len(times)):
        ax.hlines(y=2, xmin=times[i][0], xmax=times[i][1], linewidth=4, color='lavenderblush')
        
    #draw out actual mode
    if mode:
        for m in range(len(modeStart)):
            ax.hlines(y=0.5, xmin=modeStart[m], xmax=modeEnd[m], linewidth=4, color='lime')
        
    _=plt.xlabel(f'ECE Channel {ece_chn}')
    _=plt.ylabel('raw spectrogram -kHz')
    
    
    ax = fig.add_subplot(grd[3:5])
    c = ax.pcolormesh(tGood,f[:upperBound],Sxx_enhanced[:upperBound, tstart:tend],shading='auto', cmap='hot')
   # plt.xticks(np.arange(0.5, 6, 0.25))
    
    # draw out prediction
    for i in range(len(times)):
        ax.hlines(y=2, xmin=times[i][0], xmax=times[i][1], linewidth=4, color='lavenderblush')
    
    #draw out actual mode
    if mode:
        for m in range(len(modeStart)):
            ax.hlines(y=0.5, xmin=modeStart[m], xmax=modeEnd[m], linewidth=4, color='lime')
        
    plt.xticks(rotation=45)
    _=plt.xlabel('Time (s) ')
    _=plt.ylabel('enhanced spectrogram -kHz')

    # get raw predictions and plot against time; smooth fluctuations with rolling average line
    col = pd.DataFrame(rawPredictions)
    colAve = col.rolling(130).mean()
    
    ax = fig.add_subplot(grd[5:])
    # plot raw predictions vs Mode
    plt.plot(tGood, col, 'b')
    plt.plot(tGood, colAve, 'r')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Raw Prediction')

    # save figures into folder
    if mode: 
        plt.savefig(f'/scratch/gpfs/kx2561/ML/rawSpecModels/newUpdate/not_training/plots/fourPlots/visualization_prediction-MODE-shot-{shotn}-chn-{chn}.jpg')
    else:           
        plt.savefig(f'/scratch/gpfs/kx2561/ML/rawSpecModels/newUpdate/not_training/plots/fourPlots/visualization_prediction-MODELESS-shot-{shotn}-chn-{chn}.jpg')
    plt.close()

In [ ]:
# visualizations of postprocessed model predictions: mode shots
for i in tqdm(range(len(modeShots))):
    shotn = modeShots[i]
    chn = modeChns[i]
    df = pd.read_csv(f'/scratch/gpfs/kx2561/ML/rawSpecModels/newUpdate/not_training/MODE_raw30f_predictions-shot-{shotn}-chn-{chn}.csv')

    a = pickle.load(open(f'/scratch/gpfs/aj17/datasets/NT-dataset/dataset/mode/shot-{shotn}-ece-{chn}.pkl', 'rb'))
    goodIndex = a['Shot Beginning t-index']
    badIndex = a['Shot Ending t-index']
    tGood = np.array(t[goodIndex:badIndex])  # times 

    rawSpec = a['raw spectrogram']
    
    # get raw predictions on raw spectrogram
    col = makePrediction(loaded_model, rawSpec) # raw predictions
    
    # convert raw predictions to 1s and 0s
    make_pred = processPredictions(col, threshold)  
    
    # interpolation and removing isolated ones
    make_pred = filtering(make_pred, lookBefore)
    
    # remove tiny chunks of ones again after interpolation (anything <= 0.15 sec left from consideration)
    # howLong * 0.0005 s is the upper limit of the chunk (anything equal to or less will be turned into 0s)
    # 300 => 0.15 seconds
    make_pred = filteringOnes(make_pred, howLong)
    
    # converting 1s and 0s to times      
    predicted_times = tGood * np.array(make_pred)

    # find continuous durations of the mode
    durations = filtering2(predicted_times, breakTime)

    # plot postprocessed prediction on spectrogram
    plotECE(tGood, shotn, chn, 30, durations, col, True)


In [ ]:
# visualizations of postprocessed model predictions: modeless shots
for i in tqdm(range(len(modelessShots))):
    shotn = modelessShots[i]
    chn = modelessChns[i]

    a = pickle.load(open(f'/scratch/gpfs/aj17/datasets/NT-dataset/dataset/nomode/shot-{shotn}-ece-{chn}.pkl', 'rb'))
    goodIndex = a['Shot Beginning t-index']
    badIndex = a['Shot Ending t-index']
    tGood = np.array(t[goodIndex:badIndex])  # times 
    rawSpec = a['raw spectrogram']
    
    # get raw predictions on raw spectrogram
    col = makePrediction(loaded_model, rawSpec) # raw predictions

    # convert raw predictions to 1s and 0s
    make_pred = processPredictions(col, threshold)
    
    # interpolation and removing isolated ones
    make_pred = filtering(make_pred, lookBefore)
    
    # remove tiny chunks of ones again after interpolation (anything <= 0.15 sec left from consideration)
    # howLong * 0.0005 s is the upper limit of the chunk (anything equal to or less will be turned into 0s)
    # 300 => 0.15 seconds
    make_pred = filteringOnes(make_pred, howLong)
    
    # converting 1s and 0s to times      
    predicted_times = tGood * np.array(make_pred)

    # find continuous durations of the mode
    durations = filtering2(predicted_times, breakTime)
    
    # plot postprocessed prediction on spectrogram
    plotECE(tGood, shotn, chn, 30, durations, col, False)